<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Optimizing with Decision Frames  

Work in Progress 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.6"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.6"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.6 Microsoft.Extensions.Configuration.Json, 9.0.6

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.6"
#r "nuget: Microsoft.SemanticKernel, 1.56"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.6 Microsoft.SemanticKernel, 1.56.0

Using Azure OpenAI Service


### Step 2 - Decision Frames (Thinking Hats) Scenario for a Major Purchase 

> 📜 **_"The mere formulation of a problem is far more essential than its solution, which may be merely a matter of mathematical or experimental skill. To raise new questions, new possibilities, to regard old problems from a new angle requires creative imagination and marks real advances in science."_**
>
> -- <cite>Albert Einstein (Theoretical Physicist, Best known for developing the theory of relativity)</cite>  

The code below will list the potential decision frames (using thinking hats frames) that may be appropriate to form different approaches for purchasing a house. Generative AI can be used to easily "put on a hat" (approach a problem with a specific decision frame) much easier than a human decision maker.  

In [3]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions: 
List an approach for each of the six hats. 
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Absolutely! Edward de Bono’s Six Thinking Hats provide a structured framework for thoroughly exploring a complex decision like purchasing a new home. Here’s how you could use each hat in your decision-making process:

---

**1. White Hat — Information & Facts**  
*Approach:*  
- Gather objective data: home prices in preferred locations, property taxes, market trends, average time homes stay on market, mortgage rates, school ratings, commute times, and crime statistics.
- Identify what information you still need: property history, upcoming development plans in the area.
- Avoid speculation—stick to verifiable facts.

---

**2. Red Hat — Emotions & Intuition**  
*Approach:*  
- Examine your gut feelings and initial reactions: How do you feel about living in each area or house you visit?
- Acknowledge emotional needs: proximity to family, community vibe, sense of safety, pride, or contentment.
- Express and respect your partner’s or family’s feelings about particular homes, even if they c

This can be enhanced further by having GenAI suggest which of the decision frames might be the most useful for a high-stakes decision. Therefore, the prompt above can be optimized further by listing all the decision frames and then priorotizing which ones will have the most impact and influence on the house purchase decision.  

This allows the Artificial Intelligence automation to focus on the optimization of the decision process and the human on the actual decision selection and execution!

In [4]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesListPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions:
List an approach for each of the six hats in the order of impact & influence from most to least. 
From the priorotized list suggest which thinking hats (decision frames) to focus on. 
It can be one of the hats or any combination of the thinking hats. 
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesListPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Absolutely! Edward de Bono’s **Six Thinking Hats** method is a powerful framework for comprehensive decision-making. Here’s how to apply each hat to the home-buying process, prioritized by their typical impact and influence on this specific decision:

---

### 1. **White Hat (Facts & Information)**
**Approach:**  
Start by gathering all the relevant data: your budget, mortgage interest rates, local market trends, neighborhood statistics, commute times, school quality, home inspection results, and resale value projections. Well-informed decisions minimize risk.

---

### 2. **Black Hat (Caution & Risk Assessment)**
**Approach:**  
Analyze potential pitfalls and obstacles: Could there be hidden repair costs? Is there a risk of property value dropping in the area? Are there factors like future developments or zoning changes that could impact your peace or property value? List worst-case scenarios.

---

### 3. **Red Hat (Feelings, Intuition, and Emotions)**
**Approach:**  
Reflect on your

So far, (given the house purchase decision) Generative AI has helped create decision frames with the six thinking hats framework, rank them from most impact & influence from most to least and select the top 3 decision frames. The next logical step is to leverage Generative AI to apply the selected 3 decision frames for the house purchase decision.  

This can be done in two different ways:  
1) **Single Decision Prompt:** Instruct (prompt) the GenAI model "think internally" on the decision frames, ranking, selecting the top decision frames and then finally output the applied decision frames for the house purchase decision.
2) **Workflow Decision Prompts:** This approach uses multiple prompts together to arrive at a decision approach. This breaks the problem down into smaller components and executes each component in sequence. The value is that each step can be optimized individually and each intermediate output is clearly available for analysis. This is ideal for sufficiently important decision problems.  

#### Single Decision Prompt 

In [5]:
// Set the overall system prompt to behave like a Decision Intelligence assistant (persona)
var systemPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesListPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions:

<Do Not Show these steps>
INTERNALLY THINK about an approach for each of the six hats in the order of impact & influence from most to least. 
From the priorotized list INTERNALLY PROCESS which thinking hats (decision frames) to focus on. 
It can be one of the hats or any combination of the thinking hats. 
Let's call that internal list "selected decision frames"
</Do Not Show these steps>

Only output these steps to the user: 
Finally, from the internal "selected decision frames", create an approach for each for purchasing a house.  
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesListPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Certainly! Here’s how you could approach purchasing a new home using the selected decision frames from Edward de Bono’s Six Thinking Hats:

**1. Black Hat (Critical Judgment):**  
Approach: Carefully assess the risks, drawbacks, and potential issues with each property. Scrutinize factors such as structural problems, neighborhood safety, resale potential, hidden costs (maintenance, taxes, association fees), and market trends. Think about worst-case scenarios to ensure you are not overlooking potential pitfalls.

**2. White Hat (Facts & Information):**  
Approach: Gather all relevant information and data before making a decision. Research home prices in the area, recent sales, property history, school ratings, local amenities, and future urban development plans. Review financial aspects—loan options, interest rates, your credit score, and your budget. Make an evidence-based shortlist of homes.

**3. Yellow Hat (Optimism & Benefits):**  
Approach: Explore the positives of each property. C